In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.decomposition import PCA as sklearnPCA

# Make sure that caffe is on the python path:
caffe_root = '/home/lli-ms/caffe/'
pretrained_model_root = '/home/lli-ms/caffe/'

# run this line one time only!
import sys
caffePython = pretrained_model_root + 'python'
if caffePython not in sys.path:
    sys.path.insert(0, caffePython)

import caffe
# Load mean
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# Load the trained net
MODEL = 'vgg16' #'caffeNet','vgg16','vggFace' or 'faceSNN'

savePath = '../caffeModel/'+MODEL
if not os.path.exists(savePath):
    os.makedirs(savePath)
    
if MODEL == 'vgg16':
    MODEL_FILE = caffe_root +'models/VGG16/VGG_ILSVRC_16_layers_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGG16/VGG_ILSVRC_16_layers.caffemodel'
elif MODEL == 'caffeNet':
    MODEL_FILE = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
elif MODEL == 'vggFace':
    MODEL_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F.caffemodel'
    MEAN_FILE = caffe_root + 'models/VGGFACE/VGG_mean.binaryproto'
else:
    MODEL = 'faceSNN'
    MODEL_FILE = caffe_root +'models/sraonet/siamese_lecun_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/sraonet/snapshots/sraonet_lecun_gd_sub2_iter_100000.caffemodel'
    
caffe.set_device(1)
caffe.set_mode_gpu()
if not os.path.isfile(PRETRAINED_FILE):
    print("No caffemodel!!!")
elif not os.path.isfile(MODEL_FILE):
    print("No MODEL !!!")
else:
    print "Defining the net!"
    net = caffe.Net(MODEL_FILE,
                PRETRAINED_FILE,
                caffe.TEST)
# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
if MODEL != 'faceSNN':
    # subtract the dataset-mean value in each channel
    transformer.set_mean('data', mu)
for layer_name, param in net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)
transformer.set_transpose('data', (2,0,1))
# the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_raw_scale('data', 255) 
# the reference model has channels in BGR order instead of RGB
transformer.set_channel_swap('data', (2,1,0))

/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]
Defining the net!
conv1_1	(64, 3, 3, 3) (64,)
conv1_2	(64, 64, 3, 3) (64,)
conv2_1	(128, 64, 3, 3) (128,)
conv2_2	(128, 128, 3, 3) (128,)
conv3_1	(256, 128, 3, 3) (256,)
conv3_2	(256, 256, 3, 3) (256,)
conv3_3	(256, 256, 3, 3) (256,)
conv4_1	(512, 256, 3, 3) (512,)
conv4_2	(512, 512, 3, 3) (512,)
conv4_3	(512, 512, 3, 3) (512,)
conv5_1	(512, 512, 3, 3) (512,)
conv5_2	(512, 512, 3, 3) (512,)
conv5_3	(512, 512, 3, 3) (512,)
fc6	(4096, 25088) (4096,)
fc7	(4096, 4096) (4096,)
fc8	(1000, 4096) (1000,)


In [20]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('../neuralNet/VGG16/vgg16_mod.prototxt', 
                          PRETRAINED_FILE,
                          caffe.TEST)
params_full_conv = net_full_conv.params.keys()
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, \
                                                                               conv_params[conv][1].shape)

conv1_1 weights are (64, 3, 3, 3) dimensional and biases are (64,) dimensional
conv1_2 weights are (64, 64, 3, 3) dimensional and biases are (64,) dimensional
conv2_1 weights are (128, 64, 3, 3) dimensional and biases are (128,) dimensional
conv2_2 weights are (128, 128, 3, 3) dimensional and biases are (128,) dimensional
conv3_1 weights are (256, 128, 3, 3) dimensional and biases are (256,) dimensional
conv3_2 weights are (256, 256, 3, 3) dimensional and biases are (256,) dimensional
conv3_3 weights are (256, 256, 3, 3) dimensional and biases are (256,) dimensional
conv4_1 weights are (512, 256, 3, 3) dimensional and biases are (512,) dimensional
conv4_2 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv4_3 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv5_1 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv5_2 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
fc_pca weigh

In [5]:
weight_file = '/home/lli-ms/features/vgg16_conv5_2_mit_weights.csv'
bias_file = '/home/lli-ms/features/vgg16_conv5_2_mit_biases.csv'
pca_weight = np.loadtxt(weight_file, delimiter=',')
pca_bias = np.loadtxt(bias_file, delimiter = ',')
print pca_weight.shape

(766, 100352)


In [9]:
convert_weight = pca_weight[:255,:]
convert_bias = pca_bias[:255]
print convert_weight.shape, convert_bias.shape

(255, 100352) (255,)


In [11]:
# Load the original network and extract the fully connected layers' parameters.
net = caffe.Net(MODEL_FILE, 
                PRETRAINED_FILE, 
                caffe.TEST)
params = []
for layer_name, param in net.params.iteritems():
    if 'fc' not in layer_name and 'conv5_3'not in layer_name:
        params.append(layer_name)
    
# fc_params = {name: (weights, biases)}
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)


conv1_1 weights are (64, 3, 3, 3) dimensional and biases are (64,) dimensional
conv1_2 weights are (64, 64, 3, 3) dimensional and biases are (64,) dimensional
conv2_1 weights are (128, 64, 3, 3) dimensional and biases are (128,) dimensional
conv2_2 weights are (128, 128, 3, 3) dimensional and biases are (128,) dimensional
conv3_1 weights are (256, 128, 3, 3) dimensional and biases are (256,) dimensional
conv3_2 weights are (256, 256, 3, 3) dimensional and biases are (256,) dimensional
conv3_3 weights are (256, 256, 3, 3) dimensional and biases are (256,) dimensional
conv4_1 weights are (512, 256, 3, 3) dimensional and biases are (512,) dimensional
conv4_2 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv4_3 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv5_1 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional
conv5_2 weights are (512, 512, 3, 3) dimensional and biases are (512,) dimensional


In [22]:
for pr, pr_conv in zip(params, params_full_conv[:-2]):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]
    
conv_params['fc_pca'][0].flat = convert_weight.flat  # flat unrolls the arrays
conv_params['fc_pca'][1][...] = convert_bias

In [24]:
net_full_conv.save('/home/lli-ms/caffe/models/vgg16_attr/vgg16_attr_pca.caffemodel')